# Function to Read the Excel workbook with four years of data from a CRM system to create a dataframe using pandas

<h3><font color='orange'>We will read the excel workbook into a dataframe and separate date into month, day and year.</font></h3>

In [53]:
def read_and_simplify_date():
    import pandas as pd
    df = pd.read_excel("data_question.xlsx")
    df[['year','month','day']] = df['Date of Contact'].apply(lambda x: pd.Series(x.strftime("%Y,%m,%d").split(",")))
    return df

# Function to Plot a bar chart

In [54]:
def plot_bar_chart(dataframe):
    from bokeh.io import show, output_file, output_notebook
    from bokeh.models import ColumnDataSource
    from bokeh.palettes import Spectral
    from bokeh.plotting import figure

    output_notebook()
    data = dataframe
    length = data.shape[0]
    listy = list(data.columns)
    colname = listy[0]
    colvalue = listy[1]
    months = data[colname].tolist()
    counts = data[colvalue].tolist()

    sorted_months = sorted(months, key=lambda x: counts[months.index(x)])

    p = figure(x_range=sorted_months, plot_height=350, title="Bar Chart for Months vs Client Count",
           toolbar_location=None, tools="")

    p.vbar(x=months, top=counts, width=0.9)

    p.xgrid.grid_line_color = None
    p.xaxis.axis_label = "Months"
    p.yaxis.axis_label = "Client Count"
    p.y_range.start = 0
    show(p)

# Function to Plot a Pie Chart

In [55]:
def plot_pie_chart(dataframe):
    from math import pi
    import pandas as pd
    from bokeh.io import output_file, show, output_notebook
    from bokeh.palettes import Category20c
    from bokeh.plotting import figure
    from bokeh.transform import cumsum
    from bokeh.models import LabelSet, ColumnDataSource

    output_notebook()

    data = dataframe
    listy = list(data.columns)
    colname = listy[0]
    colvalue = listy[1]
    #print(data)
    length = data.shape[0]
    data['angle'] = data[colvalue]/data[colvalue].sum() * 2*pi
    data['color'] = Category20c[length]

    p = figure(plot_height=350, title="Pie Chart for Months vs Client Count", toolbar_location=None,
           tools="hover", tooltips="@colname: @colvalue", x_range=(-0.5, 1.0))

    p.wedge(x=0, y=1, radius=0.4,
        start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
        line_color="white", fill_color='color', legend=colname, source=data)

    data[colvalue] = data[colvalue].astype(str)
    data[colvalue] = data[colvalue].str.pad(35, side = "both")
    source = ColumnDataSource(data)

    labels = LabelSet(x=0, y=1, text=colvalue, level='glyph', 
                      angle=cumsum('angle', include_zero=True), source=source, render_mode='canvas')
    p.add_layout(labels)

    p.axis.axis_label=None
    p.axis.visible=False
    p.grid.grid_line_color = None

    show(p)

# Number of Clients contacted in each Month

<h3><font color='orange'>As visible from bar plot and pie chart that most number of clients (213) were contacted in month 10 i.e. </font><font color='green'>October</font></h3>

In [56]:
df_for_not_uniq = read_and_simplify_date()
df_grp_month = df_for_not_uniq.groupby('month', sort=True).count()[['Client Name']]
df_grp_month_sorted = df_grp_month.rename(columns={"Client Name": "clientCount"}).sort_values('clientCount', ascending=False).reset_index()
print (df_grp_month_sorted)
# Using functions describe above to plot a pie and bar chart
plot_pie_chart(df_grp_month_sorted)
plot_bar_chart(df_grp_month_sorted)

   month  clientCount
0     10          213
1     09          121
2     08           79
3     06           77
4     12           77
5     05           69
6     07           66
7     11           65
8     02           64
9     01           60
10    03           58
11    04           52


# Checking if one client is contacted more then once

<h3><font color='orange'> As we can see from the below results that <font color='green'>indeed</font> many clients have been contacted more then once which means that for better insights we should calculate unique number of clients calculated in each month.</font></h3>

In [57]:
df_count = read_and_simplify_date()
df_grp = df_count.groupby(['month','year','Client Name'])['Client Name'].count().reset_index(name='clientCount')
df_grp_sorted = df_grp.sort_values('clientCount', ascending=False).reset_index()
print (df_grp_sorted.head())

   index month  year                Client Name  clientCount
0    588    10  2015    Wyman, Farrell and Haag            7
1    507    10  2013          Donnelly-Champlin            6
2    540    10  2014          Eichmann and Sons            5
3    538    10  2014  Denesik, Stehr and Carter            5
4    519    10  2013   Kuhic, Stark and Kreiger            4


# Number of Unique Clients contacted in each month

<h3><font color='orange'>As visible from bar plot and pie chart that most number of unique clients (35) were contacted in month 10 i.e.</font> <font color='green'>October</font></h3>

In [58]:
df_for_uniq = read_and_simplify_date()
df_grp_month_uniq = df_for_uniq.groupby('month', sort=True).nunique()[['Client Name']]
df_grp_month_uniq_sorted = df_grp_month_uniq.rename(columns={"Client Name": "clientCount"}).sort_values('clientCount', ascending=False).reset_index()
print (df_grp_month_uniq_sorted)
plot_pie_chart(df_grp_month_uniq_sorted)
plot_bar_chart(df_grp_month_uniq_sorted)

   month  clientCount
0     10           35
1     06           34
2     05           33
3     08           33
4     09           33
5     12           32
6     02           31
7     01           30
8     11           30
9     03           29
10    07           28
11    04           26


<h4 align='center'><font color='red'>######### End of Notebook #########</font></h4>